In [2]:
import os

image_folder = "Ukr"
east_path = "text_detection/frozen_east_text_detection.pb"
min_confidence = 0.5
width = 320
height = 320

output_folder = "tmp"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [3]:
import cv2
import numpy as np
  
  

def noise_removal(image):
  import numpy as np
  kernel = np.ones((1,1),np.uint8)
  image = cv2.dilate(image, kernel, iterations=1)
  kernel = np.ones((1,1),np.uint8)
  image = cv2.erode(image, kernel, iterations=1)    
  image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel, iterations=4)
  image = cv2.medianBlur(image, 3)
  return (image)

def illumination_removal(image):
  se=cv2.getStructuringElement(cv2.MORPH_RECT , (8,8))
  bg=cv2.morphologyEx(image, cv2.MORPH_DILATE, se)
  out_gray=cv2.divide(image, bg, scale=255)
  return out_gray

def getSkewAngle(cvImage) -> float:
  # Prep image, copy, convert to gray scale, blur, and threshold
  newImage = cvImage.copy()
  gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray, (9, 9), 0)
  thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

  # Apply dilate to merge text into meaningful lines/paragraphs.
  # Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
  # But use smaller kernel on Y axis to separate between different blocks of text
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
  dilate = cv2.dilate(thresh, kernel, iterations=2)

  # Find all contours
  contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  contours = sorted(contours, key = cv2.contourArea, reverse = True)
  for c in contours:
      rect = cv2.boundingRect(c)
      x,y,w,h = rect
      cv2.rectangle(newImage,(x,y),(x+w,y+h),(0,255,0),2)

  # Find largest contour and surround in min area box
  largestContour = contours[0]
  print (len(contours))
  minAreaRect = cv2.minAreaRect(largestContour)
  cv2.imwrite("temp/boxes.jpg", newImage)
  # Determine the angle. Convert it to the value that was originally used to obtain skewed image
  angle = minAreaRect[-1]
  if angle < -45:
      angle = 90 + angle
  return -1.0 * angle
# Rotate the image around its center
def rotateImage(cvImage, angle: float):
  newImage = cvImage.copy()
  (h, w) = newImage.shape[:2]
  center = (w // 2, h // 2)
  M = cv2.getRotationMatrix2D(center, angle, 1.0)
  newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
  return newImage


In [4]:
def preprocessing_img(img):
  # convert to grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # blur
  blur = cv2.GaussianBlur(gray, (0,0), sigmaX=33, sigmaY=33)

  # divide
  divide = cv2.divide(gray, blur, scale=255)

  # otsu threshold
  thresh = cv2.threshold(divide, 160, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

  # apply morphology
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,1))
  morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
  rgb_img = cv2.cvtColor(morph, cv2.COLOR_GRAY2RGB)
  return rgb_img

In [5]:
def segment_letters(image, word_image):
    thresh = cv2.threshold(word_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 25))
    detected_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=1)
    
    cv2.imshow('Thresh', thresh)
    cv2.imshow('Detected Lines', detected_lines)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    cnts = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    letters = []
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)
        letter = word_image[y:y+h, x:x+w]
        letters.append((x, y, w, h, letter))

    return letters if letters else [(0, 0, 0, 0, word_image)]


In [6]:
from opencv_text_detection.text_detection import text_detection

from Page_to_lines import get_lines, display_lines

for file_name in os.listdir(image_folder):
  if file_name.endswith(".jpg") or file_name.endswith(".png"):
    image_path = os.path.join(image_folder, file_name)
    print(f"Processing {image_path}")

    if "_page" in file_name:
        image = cv2.imread(image_path)
        lines = get_lines(image_path, kernel_size = 17, sigma=2, theta=9, smooth_window_len=4, threshold=0.3, peak_min_distance=2)

        # Create a folder for the current image
        current_image_folder = os.path.join(output_folder, file_name[:-4])
        if not os.path.exists(current_image_folder):
            os.makedirs(current_image_folder)

        # Process each line and save it in the folder
        for idx, line in enumerate(lines):
            if not line.size == 0:  # Check if the line is not empty
                line = cv2.cvtColor(line, cv2.COLOR_GRAY2RGB)
                output_file = os.path.join(current_image_folder, f"line_{idx}.jpg")
                cv2.imwrite(output_file, line)
    else:
        data, result_img = text_detection(image_path, east_path, min_confidence, width, height)
        output_file = os.path.join(output_folder, file_name)
        cv2.imwrite(output_file, result_img)

print("Processing completed.")

Processing Ukr\Franko_page.jpg
x1= 0 , x2= 0 , Diff=  0
x1= 0 , x2= 10 , Diff=  10
x1= 10 , x2= 22 , Diff=  12
x1= 22 , x2= 51 , Diff=  29
x1= 51 , x2= 79 , Diff=  28
x1= 79 , x2= 107 , Diff=  28
x1= 107 , x2= 135 , Diff=  28
x1= 135 , x2= 163 , Diff=  28
x1= 163 , x2= 195 , Diff=  32
x1= 195 , x2= 236 , Diff=  41
x1= 236 , x2= 264 , Diff=  28
x1= 264 , x2= 292 , Diff=  28
x1= 292 , x2= 320 , Diff=  28
x1= 320 , x2= 348 , Diff=  28
x1= 348 , x2= 376 , Diff=  28
x1= 376 , x2= 404 , Diff=  28
x1= 404 , x2= 432 , Diff=  28
x1= 432 , x2= 459 , Diff=  27
x1= 459 , x2= 499 , Diff=  40
x1= 499 , x2= 532 , Diff=  33
x1= 532 , x2= 560 , Diff=  28
x1= 560 , x2= 587 , Diff=  27
x1= 587 , x2= 615 , Diff=  28
x1= 615 , x2= 642 , Diff=  27
x1= 642 , x2= 671 , Diff=  29
x1= 671 , x2= 722 , Diff=  51
Processing Ukr\Kortlyarevskiy_sun_page.jpg
x1= 0 , x2= 0 , Diff=  0
x1= 0 , x2= 18 , Diff=  18
x1= 18 , x2= 26 , Diff=  8
x1= 26 , x2= 30 , Diff=  4
x1= 30 , x2= 35 , Diff=  5
x1= 35 , x2= 71 , Diff=  36
